# COGS 108 - Capstone Project
## Group Members:
- Vincent Tierra   - A12654756
- Adrian Herrmann  - A14202007
- Lynley Yamaguchi - A12209358



In [77]:
# Imports -  These are all you need for the assignment: do not import additional packages
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind

## Part 1: Load & Clean the Data

In [78]:
# Import the data from the files into Dataframes

df_edd = pd.read_csv('FluEDD.csv')
df_hosp = pd.read_csv('FluHospitalization.csv')
df_poverty = pd.read_csv('SDPoverty.csv')
#df_income = pd.read_csv('SD_Income.csv')

In [79]:
# Replace NaN values with 0 

df_edd.fillna(value=0, inplace=True)
df_hosp.fillna(value=0, inplace=True)
df_poverty.fillna(value=0,inplace=True)

In [80]:
# Drop unnecessary columns in hospital dataframe
df_hosp.drop(['CONDITION', 'OUTCOME'], axis=1, inplace=True)

In [81]:
# Drop unnecessary columns in poverty dataframe
df_p = df_poverty.drop(['Families below poverty level','Families with children under 18 years below poverty level'], axis=1)

In [82]:
# Create Dataframe with needed columns
df_h = df_hosp[['YEAR', 'Geography', 'RegionName', 'Total_Rate']]
df_h = df_h[df_h.Geography != 'San Diego County (Actual Rate)']
df_h

,YEAR,Geography,RegionName,Total_Rate
1,2010,Central Region,REGION CENTRAL,3.922232
2,2010,Central San Diego,CENTRAL,4.227794
3,2010,Mid-City,CENTRAL,3.066694
4,2010,Southeastern San Diego,CENTRAL,4.492795
5,2010,East Region,REGION EAST,4.102715
6,2010,Alpine,EAST,0.000000
7,2010,El Cajon,EAST,0.000000
8,2010,Harbison Crest/El Cajon,EAST,5.029784
9,2010,Jamul,EAST,0.000000
10,2010,La Mesa,EAST,8.461096


In [83]:
# Merge hospital data and povert data into one Dataframe
df_hp = pd.merge(df_h,df_p,on='Geography')
df_hp

,YEAR,Geography,RegionName,Total_Rate,Population below 100% federal poverty level (FPL)
0,2010,Central Region,REGION CENTRAL,3.922232,22.9
1,2011,Central Region,REGION CENTRAL,12.974496,22.9
2,2012,Central Region,REGION CENTRAL,5.319628,22.9
3,2013,Central Region,REGION CENTRAL,26.322233,22.9
4,2010,Central San Diego,CENTRAL,4.227794,20.9
5,2011,Central San Diego,CENTRAL,12.653194,20.9
6,2012,Central San Diego,CENTRAL,4.795588,20.9
7,2013,Central San Diego,CENTRAL,23.837902,20.9
8,2010,Mid-City,CENTRAL,3.066694,26.1
9,2011,Mid-City,CENTRAL,13.480723,26.1


# Part 2: Exploratory Data Visualization

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
lx = len(df_hp['Geography']['YEAR'==2010])
ly = len(df_hp['Total_Rate']['YEAR'==2010])
xpos = np.arange(0,lx,1)
ypos = np.arange(0,ly,1)
xpos, ypos = np.meshgrid(xpos+0.25, ypos+0.25)
xpos = xpos.flatten()
ypos = ypos.flatten()
zpos = np.zeros(lx*ly)
dx = 0.5 * np.ones_like(zpos)
dy = dx.copy()
dz = df_hp.as_matrix(df_hp['Population below 100% federal poverty level (FPL)']['YEAR'==2010]).flatten()

cs = ['r', 'g', 'b', 'y', 'c'] * ly
ax.bar3d(xpos,ypos,zpos, dx, dy,dz,color=cs)

## Part 3: Exploring The Data


## Part 4: Data Analysis


## Part 5: Discussion & Conclusions

In [ ]:
# You will need the following functions for the last section of the assignment
#  You should not import any other functions
import scipy.stats as stats
from scipy.stats import normaltest
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from sklearn.decomposition import PCA

### Part 6: Testing Distributions


### Part 7: Clustering

### Part 8: Dimensionality Reduction
